In [ ]:
# Import Dependencies

In [ ]:
# Read In CSV File

In [ ]:
# Analysis: Find Average Loan Amounts Per Type Of Loan

In [ ]:
# Analysis: Find Average Loan Type Per Gender

In [ ]:
# Analysis: Find Average Loan Amount Per Education Status

In [ ]:
# Analysis: Find Average Term Rate Per Type Of Loan

In [ ]:
# Use Machine Learning to Predict The Loan Details Of A Specific Demographic